In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [0]:
from pyspark.ml.regression import LinearRegression

# Import Data

In [0]:
data = spark.read.csv('/FileStore/tables/Ecommerce_Customers.csv', inferSchema=True, header=True)

In [0]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [0]:
for item in data.head(2)[1]:
    print(item)

hduke@hotmail.com
4547 Archer CommonDiazchester, CA 06566-8576
DarkGreen
31.92627202636016
11.109460728682564
37.268958868297744
2.66403418213262
392.2049334443264


# Transform Data into Features and Target

In [0]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [0]:
data.columns

Out[14]: ['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [0]:
# Only grabbing numerical features
assembler = VectorAssembler(
    inputCols=['Avg Session Length', 'Time on App', 'Time on Website', 'Length of Membership'],
    outputCol='features'
)

In [0]:
output = assembler.transform(data)

In [0]:
output.head(1)

Out[18]: [Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [0]:
final_data = output.select('features', 'Yearly Amount Spent')

In [0]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

# Train-Test Split

In [0]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [0]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                358|
|   mean| 497.47214074499055|
| stddev|  78.42306218403508|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [0]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                142|
|   mean| 503.95769537034346|
| stddev|  81.61650532444467|
|    min|  275.9184206503857|
|    max|  744.2218671047146|
+-------+-------------------+



# Create Model

In [0]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [0]:
lr_model = lr.fit(train_data)

## Evaluate Model

In [0]:
test_results = lr_model.evaluate(test_data)

In [0]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-12.190429706035843|
| -5.546314597677849|
|  10.09923789596212|
| -4.403810689478348|
|   6.12340835816417|
|-13.579594750560034|
| -8.123819729261186|
| 2.2698815035168423|
| -6.286822277436613|
| -4.910509686714306|
|-14.861713220107049|
|  6.189444829244792|
| -7.911262075709715|
|-1.9841169389268885|
|-11.794055489441007|
|-17.900302736732613|
| -2.549880948312591|
|  6.962736624607658|
| -17.61040981309867|
|-18.318472250320838|
+-------------------+
only showing top 20 rows



In [0]:
test_results.rootMeanSquaredError

Out[29]: 10.21166135187065

In [0]:
test_results.r2

Out[30]: 0.9842345616953321

In [0]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



# Test on Unlabeled Data

In [0]:
unlabeled_data = test_data.select('features')

In [0]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.3931845423455...|
|[30.4925366965402...|
|[30.7377203726281...|
|[30.8794843441274...|
|[30.9716756438877...|
|[31.0662181616375...|
|[31.1280900496166...|
|[31.4459724827577...|
|[31.5147378578019...|
|[31.5171218025062...|
|[31.5741380228732...|
|[31.6548096756927...|
|[31.7207699002873...|
|[31.7216523605090...|
|[31.8093003166791...|
|[31.8164283341993...|
|[31.8186165667690...|
|[31.8209982016720...|
|[31.9048571310136...|
|[31.9563005605233...|
+--------------------+
only showing top 20 rows



In [0]:
predictions = lr_model.transform(unlabeled_data)

In [0]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.3931845423455...|332.11929950922945|
|[30.4925366965402...|288.01756031759237|
|[30.7377203726281...|451.68150430026776|
|[30.8794843441274...|  494.610410674333|
|[30.9716756438877...|488.51520139872855|
|[31.0662181616375...| 462.5128879582344|
|[31.1280900496166...| 565.3765064763159|
|[31.4459724827577...| 482.6070834316117|
|[31.5147378578019...|  496.099310273898|
|[31.5171218025062...|    280.8289303371|
|[31.5741380228732...| 559.2709853806939|
|[31.6548096756927...| 469.0739788983037|
|[31.7207699002873...| 546.6861955537327|
|[31.7216523605090...|349.76104357079953|
|[31.8093003166791...| 548.5659548522822|
|[31.8164283341993...|  519.022794240389|
|[31.8186165667690...|448.96855431844824|
|[31.8209982016720...| 417.7125443886057|
|[31.9048571310136...| 491.5602672359148|
|[31.9563005605233...| 565.4444039975197|
+--------------------+------------